# SVM result

In [1]:
import argparse
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import GridSearchCV,RepeatedKFold, RepeatedStratifiedKFold, KFold, cross_val_score, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import os
import pickle
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')
import seaborn as sn
from sklearn.svm import SVC
from sklearn.metrics import precision_score, auc, roc_auc_score, recall_score, confusion_matrix, accuracy_score, f1_score, average_precision_score

In [2]:
raw_data = 'C:/Users/Salvador/Modelo_COVID19/Libretas manuscrito/BCM Infectius diseases/data_final_mor.csv'
data = pd.read_csv(raw_data)
print ("Total records", data.shape[0])

Total records 11564


In [3]:
X = data.drop(['Unnamed: 0','Mortality'],axis=1).astype(int)
y = data['Mortality'].astype(int)
y = y.values.reshape(y.shape[0],1)
feature_list = X.columns
print("X:", X.shape)
print("Y:", y.shape)

X: (11564, 37)
Y: (11564, 1)


In [4]:
import joblib
SVM_loaded = joblib.load('C:/Users/Salvador/Modelo_COVID19/Libretas manuscrito/BCM Infectius diseases/PKL/modelSVM1.pkl')

In [5]:
SVM_loaded.best_params_

{'C': 1.0, 'class_weight': 'balanced', 'gamma': 'scale', 'random_state': 422}

In [6]:
print(X.shape)
print(y.shape)

(11564, 37)
(11564, 1)


In [7]:
X = X.to_numpy()

In [8]:
rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=242)

test_auc = []
test_acc = []
test_ap = [] 
test_f1 = []
test_recall = []
test_precision = []

for train_index, test_index in rskf.split(X,y): 
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  clf = SVC(**SVM_loaded.best_params_)
  clf.fit(X_train, y_train)
  yp = clf.predict(X_test) 
  test_auc.append(roc_auc_score(y_test, yp))
  test_acc.append(accuracy_score(y_test, yp))
  test_ap.append(average_precision_score(y_test, yp))
  test_f1.append(f1_score(y_test, yp))
  test_recall.append(recall_score(y_test, yp))
  test_precision.append(precision_score(y_test, yp))

In [9]:
SVM_results = pd.DataFrame({'auc':test_auc, 'acc':test_acc, 'ap':test_ap, 'f1':test_f1, 'recall':test_recall,'precision':test_precision})
SVM_results.head()

,auc,acc,ap,f1,recall,precision
0,0.750478,0.681936,0.262978,0.426791,0.845679,0.285417
1,0.768568,0.713051,0.282549,0.452145,0.845679,0.308559
2,0.744899,0.702679,0.265315,0.432343,0.803681,0.295711
3,0.757771,0.698358,0.272792,0.439807,0.840491,0.297826
4,0.770357,0.720588,0.286170,0.457143,0.839506,0.314088


In [10]:
file = 'C:/Users/Salvador/Modelo_COVID19/Libretas manuscrito/BCM Infectius diseases/result/SVM_result.csv'
SVM_results.to_csv(file,index=False)